In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import torch
resample_filter = [1, 1]
f = torch.as_tensor(resample_filter, dtype=torch.float32)
f = f.ger(f).unsqueeze(0).unsqueeze(1) / f.sum().square()
f

tensor([[[[0.2500, 0.2500],
          [0.2500, 0.2500]]]])

In [10]:
f.shape

torch.Size([1, 1, 2, 2])

In [7]:
import sys
sys.path.append(".")
from utils.layer_hook_utils import get_module_name_shapes

In [4]:
## UNet model creater
import logging
from networks_edm_RAVEN import SongUNet
def create_model_RAVEN(config):
    from networks_edm_RAVEN import SongUNet
    unet = SongUNet(in_channels=config.channels, 
                    out_channels=config.channels, 
                    num_blocks=config.layers_per_block, 
                    attn_resolutions=config.attn_resolutions, 
                    model_channels=config.model_channels, 
                    channel_mult=config.channel_mult, 
                    dropout=0.13, 
                    img_resolution=config.img_size, 
                    label_dim=0,
                    embedding_type='positional', 
                    encoder_type='standard', 
                    decoder_type='standard', 
                    augment_dim=9, 
                    channel_mult_noise=1, 
                    resample_filter=[1,1,1],  
                    stride=3, 
                    spatial_matching=config.spatial_matching
                    )
    pytorch_total_grad_params = sum(p.numel() for p in unet.parameters() if p.requires_grad)
    logging.info(f'total number of trainable parameters in the Score Model: {pytorch_total_grad_params}')
    pytorch_total_params = sum(p.numel() for p in unet.parameters())
    logging.info(f'total number of parameters in the Score Model: {pytorch_total_params}')
    return unet

[autoreload of networks_edm_RAVEN failed: Traceback (most recent call last):
  File "/n/home12/binxuwang/.local/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/n/home12/binxuwang/.local/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/n/home12/binxuwang/.local/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/n/home12/binxuwang/.local/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
  File "/n/home12/binxuwang/.local/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/n/home12/binxuwang/.local/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 349, in update_class
    if update_generi

In [9]:
from easydict import EasyDict as edict
config = edict()
config.channel_mult = [1, 2, 4,]
config.model_channels = 64
config.attn_resolutions = [9, 3,]
config.layers_per_block = 4
config.num_fid_sample = 5000
config.fid_batch_size = 1024
config.channels = 3
config.img_size = 9
config.device = "cuda"
config.sigma_min = 0.002
config.sigma_max = 80.0
config.rho = 7.0 
config.sigma_data = 0.5
config.spatial_matching = "bilinear" # "padding"
device = "cuda"

model = create_model_RAVEN(config)

In [10]:
import torch
get_module_name_shapes(model, [torch.randn(2,3,9,9), torch.randn(2,), None], deepest=4);

------------------------------------------------------------------------------------------------------------------------------------------------------
      Layer Id                                           inshape                    outshape                            Type   Module Path                      
        0                                             (2, 3, 9, 9)                (2, 3, 9, 9)                         [Input]   Image                            
        1                                                   [(2,)]                     (2, 64)           [PositionalEmbedding]   .map_noise                       
        2                                                [(2, 64)]                    (2, 256)                        [Linear]   .map_layer0                      
        3                                               [(2, 256)]                    (2, 256)                        [Linear]   .map_layer1                      
        4                           

In [4]:
from easydict import EasyDict as edict
config = edict()
config.channel_mult = [1, 2, 4]
config.model_channels = 64
config.attn_resolutions = [9, 3, 1]
config.layers_per_block = 2
config.num_fid_sample = 5000
config.fid_batch_size = 1024
config.channels = 3
config.img_size = 9
config.device = "cuda"
config.sigma_min = 0.002
config.sigma_max = 80.0
config.rho = 7.0 
config.sigma_data = 0.5
config.spatial_matching = "bilinear" # "padding"
device = "cuda"

model = create_model_RAVEN(config)

In [6]:
import torch
get_module_name_shapes(model, [torch.randn(2,3,9,9), torch.randn(2,), None], deepest=4);

------------------------------------------------------------------------------------------------------------------------------------------------------
      Layer Id                                           inshape                    outshape                            Type   Module Path                      
        0                                             (2, 3, 9, 9)                (2, 3, 9, 9)                         [Input]   Image                            
        1                                                   [(2,)]                     (2, 64)           [PositionalEmbedding]   .map_noise                       
        2                                                [(2, 64)]                    (2, 256)                        [Linear]   .map_layer0                      
        3                                               [(2, 256)]                    (2, 256)                        [Linear]   .map_layer1                      
        4                           

In [35]:
model.dec["9x9_up"]

UNetBlock(
  (norm0): GroupNorm()
  (conv0): Conv2d()
  (affine): Linear()
  (norm1): GroupNorm()
  (conv1): Conv2d()
  (skip): Conv2d()
)

In [38]:
model.dec["9x9_up"].conv0.up

True

In [39]:
model.dec["9x9_up"].conv0.fused_resample

False

In [42]:
f = model.dec["9x9_up"].conv0.resample_filter
f_pad = (f.shape[-1] - 1) // 2 if f is not None else 0

In [43]:
f_pad

1

In [6]:
import torch
model(torch.randn(1,3,9,9), torch.randn(1,), None)

tensor([[[[ 9.6287e-07, -1.5295e-06, -3.0712e-06,  2.7668e-06, -4.0174e-06,
            1.2261e-06, -8.4298e-06,  4.7294e-06,  3.3225e-07],
          [ 5.7526e-06, -2.9422e-06,  4.8267e-07, -8.8003e-06, -4.4549e-08,
           -2.6706e-05, -5.3803e-06,  1.3947e-06,  9.2550e-07],
          [ 6.1357e-06, -2.9310e-06,  6.1383e-06,  9.5705e-06, -3.6464e-06,
            3.9407e-06,  1.3022e-06, -6.7710e-06,  3.0918e-06],
          [ 1.1544e-05, -7.9482e-06, -3.0755e-06,  1.1524e-05,  6.9931e-06,
           -1.0330e-05, -1.7321e-05,  7.8962e-06, -4.4295e-06],
          [-3.8420e-08,  3.0335e-06, -4.5736e-07, -1.2166e-05, -7.0410e-06,
           -7.2965e-06,  2.4824e-06,  6.7558e-06,  8.4445e-07],
          [-3.1649e-06, -2.6774e-06,  1.4039e-06, -1.6482e-05, -1.8474e-05,
            5.9590e-06, -1.7292e-05, -7.3826e-06, -3.5822e-06],
          [-4.1022e-06, -3.1722e-06,  1.0983e-07,  2.2089e-06, -7.9987e-07,
           -4.9554e-06, -8.9188e-06,  7.3662e-07, -3.0958e-06],
          [ 3.6612e-

In [ ]:
from train_edm import create_model
from easydict import EasyDict as edict
config = edict()
config.channel_mult = [1, 2, 4, 4]
config.model_channels = 64
config.attn_resolutions = [0,1,2]
config.layers_per_block = 1
config.num_fid_sample = 5000
config.fid_batch_size = 1024
config.channels = 3
config.img_size = 9
config.device = "cuda"
config.sigma_min = 0.002
config.sigma_max = 80.0
config.rho = 7.0 
config.sigma_data = 0.5
config.spatial_matching = "bilinear"#"padding"
device = "cuda"

model = create_model(config)

In [ ]:
from train_edm import create_model
from easydict import EasyDict as edict
config = edict()
config.channel_mult = [1, 2, 4, 4]
config.model_channels = 64
config.attn_resolutions = [2,4]
config.layers_per_block = 2
config.num_fid_sample = 5000
config.fid_batch_size = 1024
config.channels = 3
config.img_size = 9
config.device = "cuda"
config.sigma_min = 0.002
config.sigma_max = 80.0
config.rho = 7.0 
config.sigma_data = 0.5
config.spatial_matching = "bilinear"#"padding"
device = "cuda"

model = create_model(config)

In [ ]:
model

SongUNet(
  (map_noise): PositionalEmbedding()
  (map_augment): Linear()
  (map_layer0): Linear()
  (map_layer1): Linear()
  (enc): ModuleDict(
    (9x9_conv): Conv2d()
    (9x9_block0): UNetBlock(
      (norm0): GroupNorm()
      (conv0): Conv2d()
      (affine): Linear()
      (norm1): GroupNorm()
      (conv1): Conv2d()
    )
    (9x9_block1): UNetBlock(
      (norm0): GroupNorm()
      (conv0): Conv2d()
      (affine): Linear()
      (norm1): GroupNorm()
      (conv1): Conv2d()
    )
    (4x4_down): UNetBlock(
      (norm0): GroupNorm()
      (conv0): Conv2d()
      (affine): Linear()
      (norm1): GroupNorm()
      (conv1): Conv2d()
      (skip): Conv2d()
    )
    (4x4_block0): UNetBlock(
      (norm0): GroupNorm()
      (conv0): Conv2d()
      (affine): Linear()
      (norm1): GroupNorm()
      (conv1): Conv2d()
      (skip): Conv2d()
      (norm2): GroupNorm()
      (qkv): Conv2d()
      (proj): Conv2d()
    )
    (4x4_block1): UNetBlock(
      (norm0): GroupNorm()
      (conv

In [5]:
model = model.to(device)

In [7]:
import torch
model(torch.randn(3, 3, 9, 9).to(device), torch.randn(3).to(device), None).shape

torch.Size([3, 3, 9, 9])

In [8]:
!pwd 

/n/home12/binxuwang/Github/mini_edm


In [9]:
!ls /n/home12/binxuwang/Github/mini_edm/exps

base_cifar10_20240130-2317	base_mnist_20240130-2207
base_cifar10_20240130-2318	base_RAVEN10_20240131-2049
base_gabor_prime_20240130-2251	base_RAVEN10_20240131-2356
base_gabor_sf_20240130-2306	base_RAVEN10_20240201-0033
base_mnist_20240129-1342	base_RAVEN10_20240201-0036
base_mnist_20240129-1406


In [10]:
!ls /n/home12/binxuwang/Github/mini_edm/exps/base_RAVEN10_20240201-0036

checkpoints  dataset_idx.pkl  samples  std.log


In [11]:
!ls /n/home12/binxuwang/Github/mini_edm/exps/base_RAVEN10_20240201-0036/samples

tensor_0.pt	 tensor_13000.pt  tensor_4000.pt  tensor_7500.pt
tensor_10000.pt  tensor_13500.pt  tensor_4500.pt  tensor_8000.pt
tensor_1000.pt	 tensor_14000.pt  tensor_5000.pt  tensor_8500.pt
tensor_10500.pt  tensor_1500.pt   tensor_500.pt   tensor_9000.pt
tensor_11000.pt  tensor_2000.pt   tensor_5500.pt  tensor_9500.pt
tensor_11500.pt  tensor_2500.pt   tensor_6000.pt
tensor_12000.pt  tensor_3000.pt   tensor_6500.pt
tensor_12500.pt  tensor_3500.pt   tensor_7000.pt


In [12]:
sample_dir = "/n/home12/binxuwang/Github/mini_edm/exps/base_RAVEN10_20240201-0036/samples"

In [27]:
import torch
epoch = 18500
samples = torch.load(f"{sample_dir}/tensor_{epoch}.pt")

In [28]:
samples[1].round().int()

tensor([[[ 0,  1, -1, -1,  0, -1,  1, -1,  2],
         [ 0, -1, -1, -1, -1, -1,  0, -1,  0],
         [ 0,  4, -1,  6,  6, -1,  1,  2,  1],
         [-1, -1,  0, -1,  7, -1,  5,  6,  5],
         [ 5, -1,  6,  7,  0,  6, -1,  6,  5],
         [ 3,  1,  2,  2, -1, -1,  5,  5,  6],
         [ 6,  0, -1,  6, -1,  4,  5,  5, -1],
         [ 4,  4,  6, -1,  1, -1,  5,  5, -1],
         [ 1,  0,  4, -1,  1,  3,  5, -1,  5]],

        [[ 0,  0, -1, -1,  3, -1,  2, -1,  8],
         [ 0, -1, -1, -1, -1, -1,  6, -1,  5],
         [ 2,  0, -1,  3,  3, -1,  3,  8,  8],
         [-1, -1,  7, -1,  6, -1,  9,  7,  1],
         [ 7, -1,  7,  6,  6,  6, -1,  4,  5],
         [ 7,  7,  7,  7, -1, -1,  4,  7,  6],
         [ 7,  6, -1,  7, -1,  7,  9,  9, -1],
         [ 6,  6,  7, -1,  7, -1,  2,  9, -1],
         [ 6,  6,  7, -1,  7,  7,  8, -1,  8]],

        [[ 7,  8, -1, -1,  8, -1,  2, -1,  7],
         [ 7, -1, -1, -1, -1, -1,  3, -1,  7],
         [ 5,  7, -1,  7,  5, -1,  5,  3,  3],
         